In [4]:
import sqlite3
import pandas as pd

In [10]:
con = sqlite3.connect("database.sqlite")
data_country = pd.read_sql_query("SELECT * from Country", con)
data_league = pd.read_sql_query("SELECT * from League", con)
data_match = pd.read_sql_query("SELECT * from Match", con)
data_player = pd.read_sql_query("SELECT * from Player", con)
data_player_att = pd.read_sql_query("SELECT * from Player_Attributes", con)
data_team = pd.read_sql_query("SELECT * from Team", con)
data_team_att = pd.read_sql_query("SELECT * from Team_Attributes", con)


# DATA PREPROCESSING

### NOTES
- home_player_1 is the id of one of the players
- The last few variables with the codes follow the convention:
    - ___H = ___ home win odds
    - ___D = ___ draw win odds
    - ___A = ___ away win odds
    - where ___ is a betting platform
    
### TODO:
- What does home_player_X1 home_player_Y1 and all others like that mean?


In [47]:
data_match['away_player_1'].dropna()

144      1.0
145      1.0
146      1.0
147      1.0
148      1.0
        ... 
25974    1.0
25975    1.0
25976    1.0
25977    1.0
25978    1.0
Name: away_player_Y1, Length: 24147, dtype: float64

In [50]:
# Start by dropping all the stuff in xtml form
match = data_match.drop(['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession'], axis = 1)
match

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
# Look for all columns with too many NA
nan_counts = match.isna().sum(axis=0)
nan_counts = pd.DataFrame({'nan_perc': nan_counts/25979})
display(nan_counts[nan_counts['nan_perc'] > 0.05].to_string())

'                 nan_perc\nhome_player_X1   0.070095\nhome_player_X2   0.070095\nhome_player_X3   0.070518\nhome_player_X4   0.070518\nhome_player_X5   0.070518\nhome_player_X6   0.070518\nhome_player_X7   0.070518\nhome_player_X8   0.070518\nhome_player_X9   0.070518\nhome_player_X10  0.070518\nhome_player_X11  0.070518\naway_player_X1   0.070518\naway_player_X2   0.070518\naway_player_X3   0.070518\naway_player_X4   0.070518\naway_player_X5   0.070518\naway_player_X6   0.070518\naway_player_X7   0.070518\naway_player_X8   0.070518\naway_player_X9   0.070557\naway_player_X10  0.070557\naway_player_X11  0.070788\nhome_player_Y1   0.070095\nhome_player_Y2   0.070095\nhome_player_Y3   0.070518\nhome_player_Y4   0.070518\nhome_player_Y5   0.070518\nhome_player_Y6   0.070518\nhome_player_Y7   0.070518\nhome_player_Y8   0.070518\nhome_player_Y9   0.070518\nhome_player_Y10  0.070518\nhome_player_Y11  0.070518\naway_player_Y1   0.070518\naway_player_Y2   0.070518\naway_player_Y3   0.070518\n

In [72]:
print(nan_counts[nan_counts['nan_perc'] > 0.1].sort_values(by= ['nan_perc'], ascending = False))

       nan_perc
PSH    0.570114
PSA    0.570114
PSD    0.570114
BSA    0.454906
BSD    0.454906
BSH    0.454906
GBA    0.454867
GBD    0.454867
GBH    0.454867
SJD    0.341892
SJH    0.341892
SJA    0.341892
IWA    0.133146
IWD    0.133146
IWH    0.133146
LBA    0.131760
LBH    0.131760
LBD    0.131760
VCH    0.131298
VCA    0.131298
VCD    0.131298
WHH    0.131183
WHA    0.131183
WHD    0.131183
BWA    0.131029
BWD    0.131029
BWH    0.131029
B365D  0.130375
B365A  0.130375
B365H  0.130375


#### EXPLORATION CONCLUSIONS
- looks like all home_player and away_player variables have over 5% missing value
- looks liek all betting websites have ober 13% missing data
- some have just under 60% missing

In [ ]:
# Make dummy variables with win loss and draw
